2.  Pessoas mais velhas fazem mais churn

In [ ]:
from google.colab import auth
import pandas as pd

# get credentials
auth.authenticate_user()
query = '''
SELECT user_id
, CASE
WHEN birth_year >= 1997 THEN 'Gen Z'
WHEN birth_year >= 1981 THEN 'Millenial'
WHEN birth_year >= 1965 THEN 'Gen X'
ELSE 'Baby Boomer'
END AS generation
, churner
FROM `e-tensor-411113.neobankclean.userdevice_clean`
'''

df = pd.read_gbq(query, project_id="e-tensor-411113")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17512 entries, 0 to 17511
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     17512 non-null  object
 1   generation  17512 non-null  object
 2   churner     17512 non-null  Int64 
dtypes: Int64(1), object(2)
memory usage: 427.7+ KB


In [ ]:
pivot = pd.pivot_table(df,
                       values=['churner','generation'],
                       index=['churner'], columns=['generation'], aggfunc='count',
                       dropna=True)

In [ ]:
pivot

user_id                      
generation Baby Boomer Gen X Gen Z Millenial
churner                                     
0                 1154  3523  1208      9022
1                  276   666   210      1453

In [ ]:
df['churner'].value_counts()

churner
0    14907
1     2605
Name: count, dtype: Int64

In [ ]:
# 276/2605 ~= 10%
# dentre os churners, 10,6% são Baby Boomers
# dentre os churners, 25,5% são Gen X
# dentre os churners, 55,8% são Millenial
# dentre os churners, 8,1% são Gen Z

# 1154/14907 ~= 8%
# dentre os not churners, 7,7% são Baby Boomers
# dentre os not churners, 23,6% são Gen X
# dentre os not churners, 60,5% são Millenial
# dentre os not churners, 8,2% são Gen Z

HO: Não há relação entre Baby Boomers e churners

H1: Há relação entre Baby Boomers e churners

In [ ]:
import scipy.stats as stats

# Teste Chi quadrado
chi_res = stats.chi2_contingency(pivot)

print(f'p-value: {chi_res[1]}')

p-value: 2.0515876957676316e-07




*   0 <= p-value <= 1

Se p-value < valor de significância: H0 é rejeitada. Fixando o valor de significância em 0.05 (valor comumente usado na literatura), temos:
*   p-value < 0.05: H0 rejeitada ⇒ H1 aceita
*   p-value=> 0.05: H0 aceita ⇒ H1 rejeitada